In [1]:
import pandas as pd

In [2]:
#load all ti matches
matches_df = pd.read_json('all_matches.json')
matches_df['match_id'].count()



55

In [3]:
def get_winner(matchid):
    match_data = matches_df[matches_df['match_id']==matchid]
    radiant_win = match_data['radiant_win'].to_string(index=False)
    winner = 1
    if radiant_win == 'True':
        winner = 0
    return winner



In [4]:
#define heroes and teams
heroes_df = pd.read_json('heroes.json')
heroes_df = heroes_df[['id','localized_name']]
heroes_df.rename(columns={"localized_name":"hero_name"},inplace=True)
heroes_df.head()


,id,hero_name
0,1,Anti-Mage
1,2,Axe
2,3,Bane
3,4,Bloodseeker
4,5,Crystal Maiden


In [5]:
#create pickbans df
picks_bans = matches_df.set_index('match_id').to_dict()['picks_bans']
picks = []
bans = []
for i in picks_bans:
    winner = get_winner(i)
    for j in picks_bans[i]:
        if j['is_pick']:
            pick = {}
            pick['match_id'] = i
            pick['order'] = j['order']
            pick['hero_id'] = j['hero_id']
            pick['team'] = j['team']
            if winner == j['team']:
                pick['win'] = 1
            else:
                pick['win'] = 0
            picks.append(pick)
        elif not j['is_pick']:
            ban = {}
            ban['match_id'] = i
            ban['order'] = j['order']
            ban['hero_id'] = j['hero_id']
            ban['team'] = j['team']
            bans.append(ban)

In [6]:
picks_df = pd.DataFrame(picks)
heroes_df['pick_count'] = heroes_df['id'].map(picks_df['hero_id'].value_counts())
wins = picks_df[['hero_id','win']].groupby(['hero_id']).sum()
wins = wins.to_dict()
heroes_df['wins'] = heroes_df['id'].map(wins['win'])
heroes_df['winrate'] = round(heroes_df['wins'] / heroes_df['pick_count'] * 100,2)

bans_df = pd.DataFrame(bans)
heroes_df['ban_count'] = heroes_df['id'].map(bans_df['hero_id'].value_counts())


heroes_df.head()

,id,hero_name,pick_count,wins,winrate,ban_count
0,1,Anti-Mage,1.0,1.0,100.00,NaN
1,2,Axe,NaN,NaN,NaN,2.0
2,3,Bane,1.0,0.0,0.00,NaN
3,4,Bloodseeker,2.0,0.0,0.00,16.0
4,5,Crystal Maiden,9.0,6.0,66.67,NaN


In [7]:
players = matches_df.set_index('match_id').to_dict()['players']
hero_stats = []
for i in players:
    for j in players[i]:
        hero_stat = {}
        hero_stat['match_id'] = j['match_id']
        hero_stat['player_name'] = j['name']
        hero_stat['account_id'] = j['account_id']
        hero_stat['hero_id'] = j['hero_id']
        hero_stat['kills'] = j['kills']
        hero_stat['deaths'] = j['deaths']
        hero_stat['assists'] = j['assists']
        hero_stat['last_hits'] = j['last_hits']
        hero_stat['xp_per_min'] = j['xp_per_min']
        hero_stat['gold_per_min'] = j['gold_per_min']
        hero_stats.append(hero_stat)
hero_stats_df = pd.DataFrame(hero_stats)
hero_stats_df.head()

,match_id,player_name,account_id,hero_id,kills,deaths,assists,last_hits,xp_per_min,gold_per_min
0,6806663153,Fata,86799300,102,0,4,7,51,361,222
1,6806663153,Quinn,221666230,52,6,4,5,300,578,526
2,6806663153,LESLÃO,87063175,137,6,4,6,178,462,378
3,6806663153,Yawar,108452107,10,2,0,1,331,647,535
4,6806663153,MSS,86726887,128,2,6,10,42,301,187


In [8]:
hero_avg_kills = hero_stats_df[['hero_id','kills']].groupby('hero_id').mean()
hero_avg_kills = hero_avg_kills.to_dict()
heroes_df['avg_kills'] = heroes_df['id'].map(hero_avg_kills['kills'])

hero_max_kills = hero_stats_df[['hero_id','kills']].groupby('hero_id').max()
hero_max_kills = hero_max_kills.to_dict()
heroes_df['max_kills'] = heroes_df['id'].map(hero_max_kills['kills'])

hero_avg_deaths = hero_stats_df[['hero_id','deaths']].groupby('hero_id').mean()
hero_avg_deaths = hero_avg_deaths.to_dict()
heroes_df['avg_deaths'] = heroes_df['id'].map(hero_avg_deaths['deaths'])

hero_max_deaths = hero_stats_df[['hero_id','deaths']].groupby('hero_id').max()
hero_max_deaths = hero_max_deaths.to_dict()
heroes_df['max_deaths'] = heroes_df['id'].map(hero_max_deaths['deaths'])

hero_avg_assists = hero_stats_df[['hero_id','assists']].groupby('hero_id').mean()
hero_avg_assists = hero_avg_assists.to_dict()
heroes_df['avg_assists'] = heroes_df['id'].map(hero_avg_assists['assists'])

hero_max_assists = hero_stats_df[['hero_id','assists']].groupby('hero_id').max()
hero_max_assists = hero_max_assists.to_dict()
heroes_df['max_assists'] = heroes_df['id'].map(hero_max_assists['assists'])

hero_avg_last_hits = hero_stats_df[['hero_id','last_hits']].groupby('hero_id').mean()
hero_avg_last_hits = hero_avg_last_hits.to_dict()
heroes_df['avg_last_hits'] = heroes_df['id'].map(hero_avg_last_hits['last_hits'])

hero_max_last_hits = hero_stats_df[['hero_id','last_hits']].groupby('hero_id').max()
hero_max_last_hits = hero_max_last_hits.to_dict()
heroes_df['max_last_hits'] = heroes_df['id'].map(hero_max_last_hits['last_hits'])

hero_avg_xpm = hero_stats_df[['hero_id','xp_per_min']].groupby('hero_id').mean()
hero_avg_xpm = hero_avg_xpm.to_dict()
heroes_df['avg_xp_per_min'] = heroes_df['id'].map(hero_avg_xpm['xp_per_min'])

hero_avg_gpm = hero_stats_df[['hero_id','gold_per_min']].groupby('hero_id').mean()
hero_avg_gpm = hero_avg_gpm.to_dict()
heroes_df['avg_gold_per_min'] = heroes_df['id'].map(hero_avg_gpm['gold_per_min'])

heroes_df.head()

,id,hero_name,pick_count,wins,winrate,ban_count,avg_kills,max_kills,avg_deaths,max_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min
0,1,Anti-Mage,1.0,1.0,100.00,NaN,7.000000,7.0,6.000000,6.0,13.000000,13.0,471.000000,471.0,927.000000,700.000000
1,2,Axe,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Bane,1.0,0.0,0.00,NaN,0.000000,0.0,6.000000,6.0,2.000000,2.0,41.000000,41.0,237.000000,186.000000
3,4,Bloodseeker,2.0,0.0,0.00,16.0,9.000000,14.0,4.000000,4.0,10.000000,12.0,396.500000,466.0,698.000000,601.500000
4,5,Crystal Maiden,9.0,6.0,66.67,NaN,2.111111,3.0,5.666667,13.0,15.666667,24.0,66.222222,116.0,402.777778,292.555556


In [9]:
heroes_morepicks_df = heroes_df[heroes_df['pick_count']>=5]

In [37]:
ti_teams_df= pd.read_json('teams.json')
ti_teams = list(ti_teams_df['team_id'])
players_df = pd.read_json('proPlayers.json')
players_df = players_df[['account_id','personaname','name','team_id','team_name','team_tag']]
players_df = players_df[players_df['team_id'].isin(ti_teams)]
players_df.head()


,account_id,personaname,name,team_id,team_name,team_tag
30,16497807,tOfu,tOfu,8599101,Gaimin Gladiators,GG
52,25907144,Let's Go,Cr1t-,39,Evil Geniuses,EG
146,54580962,insania,iNsania,2163,Team Liquid,Liquid
206,72312627,kimchi,MATUMBAMAN,2163,Team Liquid,Liquid
211,73562326,middus diffus,zai,2163,Team Liquid,Liquid


In [65]:
player_avg_kills = hero_stats_df[['account_id','kills']].groupby('account_id').mean()
player_avg_kills = player_avg_kills.to_dict()
players_df['avg_kills'] = players_df['account_id'].map(player_avg_kills['kills'])

player_max_kills = hero_stats_df[['account_id','kills']].groupby('account_id').max()
player_max_kills = player_max_kills.to_dict()
players_df['max_kills'] = players_df['account_id'].map(player_max_kills['kills'])

player_avg_deaths = hero_stats_df[['account_id','deaths']].groupby('account_id').mean()
player_avg_deaths = player_avg_deaths.to_dict()
players_df['avg_deaths'] = players_df['account_id'].map(player_avg_deaths['deaths'])

player_max_deaths = hero_stats_df[['account_id','deaths']].groupby('account_id').max()
player_max_deaths = player_max_deaths.to_dict()
players_df['max_deaths'] = players_df['account_id'].map(player_max_deaths['deaths'])

player_avg_assists = hero_stats_df[['account_id','assists']].groupby('account_id').mean()
player_avg_assists = player_avg_assists.to_dict()
players_df['avg_assists'] = players_df['account_id'].map(player_avg_assists['assists'])

player_max_assists = hero_stats_df[['account_id','assists']].groupby('account_id').max()
player_max_assists = player_max_assists.to_dict()
players_df['max_assists'] = players_df['account_id'].map(player_max_assists['assists'])

player_avg_last_hits = hero_stats_df[['account_id','last_hits']].groupby('account_id').mean()
player_avg_last_hits = player_avg_last_hits.to_dict()
players_df['avg_last_hits'] = players_df['account_id'].map(player_avg_last_hits['last_hits'])

player_max_last_hits = hero_stats_df[['account_id','last_hits']].groupby('account_id').max()
player_max_last_hits = player_max_last_hits.to_dict()
players_df['max_last_hits'] = players_df['account_id'].map(player_max_last_hits['last_hits'])

player_avg_xpm = hero_stats_df[['account_id','xp_per_min']].groupby('account_id').mean()
player_avg_xpm = player_avg_xpm.to_dict()
players_df['avg_xp_per_min'] = players_df['account_id'].map(player_avg_xpm['xp_per_min'])

player_avg_gpm = hero_stats_df[['account_id','gold_per_min']].groupby('account_id').mean()
player_avg_gpm = player_avg_gpm.to_dict()
players_df['avg_gold_per_min'] = players_df['account_id'].map(player_avg_gpm['gold_per_min'])

player_max_gpm = hero_stats_df[['account_id','gold_per_min']].groupby('account_id').max()
player_max_gpm = player_max_gpm.to_dict()
players_df['max_gold_per_min'] = players_df['account_id'].map(player_max_gpm['gold_per_min'])

player_unique_heroes = hero_stats_df[['account_id','hero_id']].groupby('account_id').count()
player_unique_heroes = player_unique_heroes.to_dict()
players_df['unique_heroes'] = players_df['account_id'].map(player_unique_heroes['hero_id'])

players_df.head()

,account_id,personaname,name,team_id,team_name,team_tag,avg_kills,max_kills,avg_deaths,max_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min,max_gold_per_min,unique_heroes
30,16497807,tOfu,tOfu,8599101,Gaimin Gladiators,GG,3.000000,7.0,5.666667,8.0,17.166667,38.0,82.000000,105.0,488.500000,307.666667,395.0,6.0
52,25907144,Let's Go,Cr1t-,39,Evil Geniuses,EG,5.333333,12.0,3.333333,6.0,17.666667,23.0,54.666667,79.0,505.000000,321.833333,376.0,6.0
146,54580962,insania,iNsania,2163,Team Liquid,Liquid,2.833333,7.0,5.666667,11.0,14.833333,24.0,84.500000,229.0,382.500000,304.333333,450.0,6.0
206,72312627,kimchi,MATUMBAMAN,2163,Team Liquid,Liquid,4.833333,10.0,3.833333,5.0,10.833333,19.0,426.166667,1009.0,592.500000,592.166667,908.0,6.0
211,73562326,middus diffus,zai,2163,Team Liquid,Liquid,5.000000,9.0,5.166667,9.0,11.833333,20.0,201.333333,356.0,556.833333,426.833333,564.0,6.0


# Hero with the most picked:

In [10]:
hero_picks = heroes_df[['hero_name','pick_count']]
pick_max = hero_picks['pick_count'].max()
hero_picks[hero_picks['pick_count']==pick_max]


,hero_name,pick_count
50,Leshrac,21.0
122,Primal Beast,21.0


# Hero with the most banned:

In [11]:
ban_picks = heroes_df[['hero_name','ban_count']]
ban_max = ban_picks['ban_count'].max()
ban_picks[ban_picks['ban_count']==ban_max]

,hero_name,ban_count
63,Batrider,38.0


# Hero with the highest win rate:

In [12]:
winrate_df = heroes_morepicks_df[['hero_name','winrate']]
win_max = winrate_df['winrate'].max()
winrate_df[winrate_df['winrate']==win_max]

,hero_name,winrate
62,Jakiro,83.33


# Hero with the highest average kill:

In [13]:
hero_kill_avg = heroes_morepicks_df[['hero_name','avg_kills']]
avg_kill_max = hero_kill_avg['avg_kills'].max()
hero_kill_avg[hero_kill_avg['avg_kills']==avg_kill_max]



,hero_name,avg_kills
68,Ursa,11.0


# Hero with the highest average assist:

In [14]:
hero_asst_avg = heroes_morepicks_df[['hero_name','avg_assists']]
avg_asst_max = hero_asst_avg['avg_assists'].max()
hero_asst_avg[hero_asst_avg['avg_assists']==avg_asst_max]

,hero_name,avg_assists
73,Silencer,18.0


# Hero with the lowest average death:

In [15]:
hero_deaths_avg = heroes_morepicks_df[['hero_name','avg_deaths']]
avg_deaths_min = hero_deaths_avg['avg_deaths'].min()
hero_deaths_avg[hero_deaths_avg['avg_deaths']==avg_deaths_min]

,hero_name,avg_deaths
59,Broodmother,2.5


# Hero with the highest average last hit:

In [16]:
hero_lh_avg = heroes_morepicks_df[['hero_name','avg_last_hits']]
avg_lh_max = hero_lh_avg['avg_last_hits'].max()
hero_lh_avg[hero_lh_avg['avg_last_hits']==avg_lh_max]

,hero_name,avg_last_hits
87,Naga Siren,567.857143


# Hero with the highest average xp per minute:

In [17]:
hero_xpm_avg = heroes_morepicks_df[['hero_name','avg_xp_per_min']]
avg_xpm_max = hero_xpm_avg['avg_xp_per_min'].max()
hero_xpm_avg[hero_xpm_avg['avg_xp_per_min']==avg_xpm_max]

,hero_name,avg_xp_per_min
13,Pudge,762.25


# Hero with the most kills in a game:

In [18]:
hero_max_kills = heroes_morepicks_df[['hero_name','max_kills']]
maxkills = hero_max_kills['max_kills'].max()
hero_max_kills[hero_max_kills['max_kills']==maxkills]

,hero_name,max_kills
50,Leshrac,22.0


# Hero with the most last hits in a game:

In [19]:
hero_max_lh = heroes_morepicks_df[['hero_name','max_last_hits']]
maxlh = hero_max_lh['max_last_hits'].max()
hero_max_lh[hero_max_lh['max_last_hits']==maxlh]

,hero_name,max_last_hits
33,Sniper,1058.0


# Player with the highest kill average:

In [39]:
player_kill_avg = players_df[['name','avg_kills']]
player_avg_kill_max = player_kill_avg['avg_kills'].max()
player_kill_avg[player_kill_avg['avg_kills']==player_avg_kill_max]

,name,avg_kills
723,Somnus,11.0


# Player with the highest assist average:

In [40]:
player_assist_avg = players_df[['name','avg_assists']]
player_avg_assist_max = player_assist_avg['avg_assists'].max()
player_assist_avg[player_assist_avg['avg_assists']==player_avg_assist_max]

,name,avg_assists
52,Cr1t-,17.666667


# Player with the lowest death average:

In [41]:
player_death_avg = players_df[['name','avg_deaths']]
player_avg_death_max = player_death_avg['avg_deaths'].min()
player_death_avg[player_death_avg['avg_deaths']==player_avg_death_max]

,name,avg_deaths
1056,Nightfall 凛,1.166667


# Player with the highest last hit average:

In [42]:
player_last_hit_avg = players_df[['name','avg_last_hits']]
player_avg_last_hit_max = player_last_hit_avg['avg_last_hits'].max()
player_last_hit_avg[player_last_hit_avg['avg_last_hits']==player_avg_last_hit_max]

,name,avg_last_hits
2454,JACKBOYS,610.5


# Player with the highest GPM average:

In [43]:
player_gold_per_min_avg = players_df[['name','avg_gold_per_min']]
player_avg_gold_per_min_max = player_gold_per_min_avg['avg_gold_per_min'].max()
player_gold_per_min_avg[player_gold_per_min_avg['avg_gold_per_min']==player_avg_gold_per_min_max]

,name,avg_gold_per_min
2454,JACKBOYS,729.5


# Player with the most kills:

In [44]:
player_kill_max = players_df[['name','max_kills']]
player_max_kill_max = player_kill_max['max_kills'].max()
player_kill_max[player_kill_max['max_kills']==player_max_kill_max]

,name,max_kills
723,Somnus,22.0


# Player with the most assists:

In [45]:
player_assist_max = players_df[['name','max_assists']]
player_max_assist_max = player_assist_max['max_assists'].max()
player_assist_max[player_assist_max['max_assists']==player_max_assist_max]

,name,max_assists
1418,TIMS,39.0


# Player with the most last hits:

In [46]:
player_last_hit_max = players_df[['name','max_last_hits']]
player_max_last_hit_max = player_last_hit_max['max_last_hits'].max()
player_last_hit_max[player_last_hit_max['max_last_hits']==player_max_last_hit_max]

,name,max_last_hits
2454,JACKBOYS,1058.0


# Player with the most GPM:

In [49]:
player_gold_per_min_max = players_df[['name','max_gold_per_min']]
player_max_gold_per_min_max = player_gold_per_min_max['max_gold_per_min'].max()
player_gold_per_min_max[player_gold_per_min_max['max_gold_per_min']==player_max_gold_per_min_max]

,name,max_gold_per_min
909,dyrachyo,995.0


# Player with the most different heroes:

In [68]:
player_unique_heroes = players_df[['name','unique_heroes']]
player_unique_heroes_max= player_unique_heroes['unique_heroes'].max()
player_unique_heroes[player_unique_heroes['unique_heroes']==player_unique_heroes_max]

,name,unique_heroes
30,tOfu,6.0
52,Cr1t-,6.0
146,iNsania,6.0
206,MATUMBAMAN,6.0
211,zai,6.0
227,Boxi,6.0
303,MSS,6.0
311,天鸽,6.0
325,Fata,6.0
348,LESLÃO,6.0
